# Preprocessing phase

In [ ]:
import sys
import os

import pandas as pd

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from source.utils.dataHandler import DataHandler

In [ ]:
data_handler = DataHandler('../Data/export.csv')

In [ ]:
data_handler.preprocess_data()

In [ ]:
data_handler.get_data().head()

Which quarter do you prefer?

You have the following choices:
- Gen-Mar
- Apr-Giu
- Lug-Set
- Ott-Dic

In [ ]:
quarter = 'Gen-Mar'

data_handler.filter_month(quarter)

For the implementation of a recommendation system, we must know who is the client.

The only feature that provides this information is tessera, but the number of fidelity customers is very low

You can select chose between fidelity’s number and receipt’s code to index the recommendation system

In [ ]:
index = 'tessera' # or index = 'id_sc'
data_handler.process(index)